**source study** : https://github.com/wasit7/dsmde/blob/main/example/01_social_media.ipynb

In [176]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

def get_soup(url,pages) :
    
    response2 = requests.get(url)
    soup2 = BeautifulSoup(response2.content, 'html.parser')
    
    try:
        abstract = soup2.select_one('#Abs1-content').text
        title = soup2.find('h1').text 
        names = [i['content'].replace(',','') for i in soup2.find_all('meta', {'name': 'dc.creator'})]
        date = soup2.select_one('#main-content > main > article > div.c-article-header > ul.c-article-identifiers > li:nth-child(3) > a > time').text
        return {'pages':pages,'title':title,'names':names[0],'date':date,'abstract':abstract,'url':url}
    except:
        return {'pages':pages,'title':None,'names':None,'date':None,'abstract':None,'url':url}

In [186]:
pages = 30
dataset = []
for i in range(1,pages+1):
    response = requests.get(f"https://health-policy-systems.biomedcentral.com/articles?searchType=journalSearch&sort=PubDate&page={i}")
    soup = BeautifulSoup(response.content, 'html.parser')
    urls_research = [ f"https://health-policy-systems.biomedcentral.com{i.find('a')['href']}" for i in soup.select_one('ol.c-listing').find_all('h3')]
    for j in tqdm(urls_research, desc=f'get abstract pages {i}', unit="iteration") :
        dataset.append(get_soup(j,i+1))

get abstract pages 30: 100%|████████████████████████████████████████████████████| 11/11 [00:11<00:00,  1.00s/iteration]


In [188]:
df = pd.DataFrame(dataset)
df.head(5)

,pages,title,names,date,abstract,url
0,2,Assessing unConventional Evidence (ACE) tool: ...,Lewin Simon,02 January 2024,BackgroundWhen deciding whether to implement a...,https://health-policy-systems.biomedcentral.co...
1,2,"""Research ends with publication"": a qualitativ...",Morankar Sudhakar,02 January 2024,BackgroundDecision-making about the design and...,https://health-policy-systems.biomedcentral.co...
2,2,A scoping review of the globally available too...,Mrklas Kelly J.,22 December 2023,BackgroundHealth research partnership approach...,https://health-policy-systems.biomedcentral.co...
3,2,Assessing the impact of an evidence- and conse...,Wabnitz Katharina,19 December 2023,"IntroductionDuring the COVID-19 pandemic, deci...",https://health-policy-systems.biomedcentral.co...
4,2,Opportunities to improve the adoption of healt...,Larose Hugo,19 December 2023,ObjectivesPatient’s health-related quality of ...,https://health-policy-systems.biomedcentral.co...


In [192]:
new_pages = df['pages'] - 1

In [193]:
df['pages'] = new_pages

In [195]:
df.to_excel("../DATASET/bmc.xlsx")  